# torch `nn.Embedding`

In [2]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Playdata/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Playdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 사전학습된 임베딩을 사용하지 않는 경우

In [3]:
sentences = [          
    'nice great best amazing',  # 긍정 문장 예시
    'stop lies',                # 부정/비판 문장 예시
    'pitiful nerd',             # 부정 문장 예시
    'excellent work',           # 긍정 문장 예시
    'supreme quality',          # 긍정 문장 예시
    'bad',                      # 부정 문장 예시
    'highly respectable'        # 긍정 문장 예시
]                               # 분류 모델에 넣을 입력 문장 리스트(list[str])
labels = [1, 0, 0, 1, 1, 0, 1]  # 각 문장에 대한 이진 라벨(1=긍정, 0=부정)

In [4]:
# 토큰화
from nltk.tokenize import word_tokenize

tokenized_sentences = [word_tokenize(sent) for sent in sentences]    # 각 문장을 토큰 리스트(list(list[str]))로 변환
tokenized_sentences

[['nice', 'great', 'best', 'amazing'],
 ['stop', 'lies'],
 ['pitiful', 'nerd'],
 ['excellent', 'work'],
 ['supreme', 'quality'],
 ['bad'],
 ['highly', 'respectable']]

In [5]:
# 단어 사전 생성 + 정수 인코딩
from collections import Counter

tokens = [token for sent in tokenized_sentences for token in sent]  # 문장 리스트를 평탄화하여 전체 토큰 리스트 생성
word_counts = Counter(tokens)  # 전체 토큰 등장 빈도 계산
print(word_counts)  # 토큰별 빈도 딕셔너리 형태

word_to_index = {word: index + 2 for index, word in enumerate(tokens)}  # 토큰을 순서대로 인덱싱(+2 : 특수토큰용)
word_to_index['<PAD>'] = 0    # 패딩 토큰 (길이 맞추기용)
word_to_index['<UNK>'] = 1    # OOV 토큰 (처리 불가 단어 대체)
word_to_index = dict(sorted(word_to_index.items(), key=lambda x: x[1]))  # 인덱스를 기준으로 정렬
print(word_to_index)  # 단어 -> 인덱스 사전

vocab_size = len(word_to_index)  # 전체 어휘 수 (특수토큰 포함)
vocab_size

Counter({'nice': 1, 'great': 1, 'best': 1, 'amazing': 1, 'stop': 1, 'lies': 1, 'pitiful': 1, 'nerd': 1, 'excellent': 1, 'work': 1, 'supreme': 1, 'quality': 1, 'bad': 1, 'highly': 1, 'respectable': 1})
{'<PAD>': 0, '<UNK>': 1, 'nice': 2, 'great': 3, 'best': 4, 'amazing': 5, 'stop': 6, 'lies': 7, 'pitiful': 8, 'nerd': 9, 'excellent': 10, 'work': 11, 'supreme': 12, 'quality': 13, 'bad': 14, 'highly': 15, 'respectable': 16}


17

In [6]:
# 정수 인코딩 함수
# : 토큰화된 문장 리스트를 단어 → 인덱스 사전을 이용해
#   정수 시퀀스(list[list[int]])로 변환
def texts_to_sequences(sentences, word_to_index):
    sequences = []                                  # 전체 문장 시퀀스를 저장할 리스트

    for sent in sentences:                          # 문장 단위로 반복
        sequence = []                               # 현재 문장의 정수 시퀀스

        for token in sent:                          # 문장 내 토큰(단어) 단위 반복
            if token in word_to_index:              # 단어가 사전에 존재하면
                sequence.append(word_to_index[token])  # 해당 단어 인덱스 추가
            else:
                sequence.append(word_to_index['<UNK>'])  # 사전에 없으면 UNK 토큰 인덱스 추가
        
        sequences.append(sequence)                  # 문장 하나의 시퀀스를 결과에 저장
    
    return sequences                                # 전체 문장 정수 시퀀스 반환


sequences = texts_to_sequences(                     # 토큰화된 문장들을 정수 시퀀스로 변환
    tokenized_sentences,
    word_to_index
)
sequences                                           # 변환 결과 확인


[[2, 3, 4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14], [15, 16]]

In [7]:
import numpy as np

# 서로 다른 길이의 정수 시퀀스를 0(<PAD>)으로 채우거나 잘라내 (문장수, maxlen) 형태에 맞춰주는 함수
def pad_sequences(sequences, maxlen):
    padded_sequences = np.zeros((len(sequences), maxlen), dtype=int)  # (문장수 x maxlen) 크기의 0 패딩 배열
    
    for index, seq in enumerate(sequences):  # 각 문장 시퀀스 순회
        padded_sequences[index, :len(seq)] = seq[:maxlen]  # 앞에서부터 시퀀스 채운다. 길면 maxlen까지만 채워 자른다.
    
    return padded_sequences  # 패딩 작업 완료된 2D 배열

padded_sequences = pad_sequences(sequences, maxlen=4)  # 모든 문장 길이 4로 패딩/자르기
padded_sequences  # (문장 수, 4) 형태

array([[ 2,  3,  4,  5],
       [ 6,  7,  0,  0],
       [ 8,  9,  0,  0],
       [10, 11,  0,  0],
       [12, 13,  0,  0],
       [14,  0,  0,  0],
       [15, 16,  0,  0]])

In [8]:
padded_sequences.shape

(7, 4)

In [9]:
# Pytorch 텍스트 분류 모델 : Embedding + RNN + Linear로 이진 분류(logit) 출력
import torch
import torch.nn as nn           # 신경망 레이어
import torch.optim as optim     # 옵티마이저(활성화함수)
from torch.utils.data import DataLoader, TensorDataset  # 배치 로더 / 데이터셋 유틸

class SimpleNet(nn.Module):
    # 정수 시퀀스를 임베딩 -> RNN -> 선형층으로 처리해 이진 분류 logit(1개)를 출력
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()                    # nn.Module 초기화
        self.embedding = nn.Embedding(        # 단어 ID를 밀집 벡터로 변환하는 임베딩 층
            num_embeddings = vocab_size,      # 단어 사전 크기 (어휘 수)
            embedding_dim = embedding_dim,    # 임베딩 차원
            padding_idx = 0                   # PAD(0) 인덱스는 0 그대로 사용
        )
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)    # 입력(배치, 길이, 차원) 형태의 RNN
        self.out = nn.Linear(hidden_size, 1)     # 마지막 은닉 상태를 1차원 logit으로 변환

    def forward(self, x):
        embedded = self.embedding(x)     # (batch, seq_len) -> (batch, seq_len, embedding_dim)
        out, h_n = self.rnn(embedded)    # h_n: (num_layers*directions, batch, hidden_size)
        out = self.out(h_n.squeeze(0))   # (batch_size, hidden_size) -> (batch, 1)
        return out  # 출력 : 시그모이드 전 logit(확률이 아님)
    
embedding_dim = 100  # 단어 벡터 차원 설정
model = SimpleNet(vocab_size, embedding_dim, hidden_size=16)  # 어휘 크기 / 임베딩 차원/ 은닉크기로 모델 생성
model

SimpleNet(
  (embedding): Embedding(17, 100, padding_idx=0)
  (rnn): RNN(100, 16, batch_first=True)
  (out): Linear(in_features=16, out_features=1, bias=True)
)

In [10]:
%pip install torchinfo

Note: you may need to restart the kernel to use updated packages.


In [11]:
from torchinfo import summary  # 모델 구조를 표 형태로 요약

summary(model)  # model의 레이어 구성 / 파라미터 수를 요약

Layer (type:depth-idx)                   Param #
SimpleNet                                --
├─Embedding: 1-1                         1,700
├─RNN: 1-2                               1,888
├─Linear: 1-3                            17
Total params: 3,605
Trainable params: 3,605
Non-trainable params: 0

In [12]:
# 임베딩 가중치 확인 : 학습 전/후 Embedding 테이블과 단어별 벡터 조회
import pandas as pd

# 학습 전 임베딩 벡터
wv = model.embedding.weight.data  # Embedding 층의 가중치 행렬(단어ID x 임베딩 차원) 추출
print(wv.shape)  # (vocab_size, embedding_dim)

# 특정 단어 벡터
vocab = word_to_index.keys()    # 단어사전에서 단어만 뽑아온다.
pd.DataFrame(wv, index=vocab)

torch.Size([17, 100])


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
<PAD>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
<UNK>,0.288928,1.607660,-0.056371,-0.469463,0.469999,-0.649331,1.339970,0.047787,1.187581,-0.378604,...,1.163199,-0.957975,-0.136281,0.781986,1.577181,-0.096898,1.777398,-0.315396,-0.166276,-0.431706
nice,-0.375107,-1.502437,-0.923492,-0.126488,0.362706,0.342689,-1.466642,1.847192,-0.179850,-0.388620,...,0.375694,0.324467,1.260270,0.276454,0.188943,1.352863,-0.998862,-0.222001,-0.426115,-1.941638
great,-2.070317,0.062140,-0.049227,-0.107032,-0.310520,1.134613,1.739478,-0.535167,-0.148535,0.059957,...,-0.768177,1.688066,0.556124,-2.068017,-1.331687,-0.080085,-0.362097,-1.092865,1.689988,0.510178
best,-0.734108,-0.694109,0.215780,0.343054,2.067814,-1.004351,-0.709888,-1.500188,0.051568,0.479909,...,-1.976840,1.053008,0.526363,-0.673199,0.777457,-0.028395,-0.826402,-0.258068,0.024702,0.888323
amazing,0.110546,0.140450,-1.180040,-0.194728,-0.007513,0.135330,-0.224973,0.292712,1.269658,-1.625910,...,-2.258460,0.843542,-0.818338,0.126159,0.631749,0.263715,-0.829794,-1.229905,-1.592322,0.011544
stop,1.161571,-0.293670,-0.926437,-0.171423,0.147151,-0.434187,0.556416,-2.125310,0.109032,-0.475795,...,-0.337052,-0.262313,0.527860,-1.155204,-0.020593,1.073358,0.721517,-1.528801,2.115555,0.229224
lies,-0.550596,-0.910341,0.699479,-2.037829,-0.102543,-0.621193,-0.988834,2.027308,1.513137,0.382081,...,-0.364772,0.934649,-1.553773,-0.259093,1.016199,0.063753,2.199706,-1.115450,-1.433277,0.827748
pitiful,-0.255950,0.279579,0.929856,1.384931,0.032164,0.319030,0.105520,0.527916,0.662397,1.087102,...,-0.722323,0.617160,1.415717,-0.759201,0.341842,1.224626,1.466413,-0.349043,-0.000191,-1.460817
nerd,1.075786,0.115069,-0.544386,-0.771980,0.644482,1.079942,-0.767168,-0.080798,0.439931,-1.540084,...,1.107514,0.151260,-1.319436,1.312837,1.290813,1.624247,0.686019,0.273804,0.775039,0.060528


In [13]:
# Pytorch 학습 준비 : 텐서 변환 -> DataLoader 구성 -> 손실함수/옵티마이저 설정
X = torch.tensor(padded_sequences, dtype=torch.long)      # 입력 시퀀스(정수 ID)를 LongTensor로 변환
y = torch.tensor(labels, dtype=torch.float).unsqueeze(1)  # 라벨을 float으로 변환 후 (N,) -> (N, 1)로 차원 맞춤

dataset = TensorDataset(X, y)  # (X, y) 쌍을 Dataset 객체로 묶음
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)  # 배치 단위로 섞어서 공급하는 로더

criterion = nn.BCEWithLogitsLoss()  # 출력 logit과 정답(0/1)로 이진분류 손실 계산 (시그모이드 포함)
optimizer = optim.Adam(model.parameters(), lr=0.005)  # 모델 파라미터는 Adam으로 업데이트

BCEWithLogitsLoss을 사용할 때에는 모델 출력이 Sigmoid를 거치지 않은 logit이어야 한다.

In [14]:
# 학습 루프 : 미니배치 단위로 20 epoch 학습하며 평균 손실 출력
for epoch in range(20):
    epoch_loss = 0    # 손실 누적

    for x_batch, y_batch in dataloader:    # 미니배치 단위로 (X, y) 가져오기
        optimizer.zero_grad()              # 이전 배치 기울기 초기화
        output = model(x_batch)            # 순전파로 logit 계산
        loss = criterion(output, y_batch)  # 예측 logit과 정답으로 손실 계산
        loss.backward()                    # 역전파로 기울기 계산
        optimizer.step()                   # 파라미터 업데이트

        epoch_loss += loss.item()  # 배치손실을 float으로 누적
    
    print(f"Epoch {epoch + 1}: Loss {epoch_loss / len(dataloader)}")  # epoch별 평균 손실 계산

Epoch 1: Loss 0.7058625668287277
Epoch 2: Loss 0.5749433189630508
Epoch 3: Loss 0.5192097276449203
Epoch 4: Loss 0.4127691611647606
Epoch 5: Loss 0.34768517687916756
Epoch 6: Loss 0.25201917067170143
Epoch 7: Loss 0.18153125792741776
Epoch 8: Loss 0.1340364422649145
Epoch 9: Loss 0.10082214698195457
Epoch 10: Loss 0.07123453728854656
Epoch 11: Loss 0.05424548406153917
Epoch 12: Loss 0.042618256993591785
Epoch 13: Loss 0.035117856692522764
Epoch 14: Loss 0.029273396357893944
Epoch 15: Loss 0.024098847527056932
Epoch 16: Loss 0.022073124069720507
Epoch 17: Loss 0.01889193383976817
Epoch 18: Loss 0.017635498195886612
Epoch 19: Loss 0.01556430128403008
Epoch 20: Loss 0.014929302269592881


In [15]:
# 평가 / 예측 : 학습된 모델로 확률 -> 0/1 예측값 생성 후 정답과 비교
model.eval()                        # 평가 모드
with torch.no_grad():               # 기울기 계산 비활성화
    output = model(X)               # 전체 샘플에 대한 예측 logit 계산
    prob = torch.sigmoid(output)    # logit에 0~1 확률로 변환
    pred = (prob >= 0.5).int()      # 임계값 0.5 기준으로 이진 분류(0/1) 예측값 생성

print(labels)
print(pred.squeeze().detach().numpy())  # 예측라벨을 1차원 numpy 배열로 변환

[1, 0, 0, 1, 1, 0, 1]
[1 0 0 1 1 0 1]


## 사전학습된 임베딩을 사용하는 경우

In [16]:
from gensim.models import KeyedVectors

model_wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
model_wv.vectors.shape

(3000000, 300)

In [17]:
# 임베딩 매트릭스 초기화 : 사전학습 벡터로 Embedding 레이어를 채우기 위한 준비
print(len(word_to_index))  # 어휘 크기 (vocab_size) 확인

# (vocab_size, embedding_dim) 크기의 0 행렬 생성
embedding_matrix = np.zeros((len(word_to_index), model_wv.vectors.shape[1]))
print(embedding_matrix.shape)

17
(17, 300)


In [ ]:
# 사전학습 임베딩 매핑 : 내 단어사전을 GoogleNews 벡터로 채워 embedding_matrix 구성
# model_wv.key_to_index['bad']  # 'bad'의 내부 인덱스 확인 (706)
# model_wv.vectors[240]         # 특정 인덱스 벡터 직접 조회

# 단어가 사전학습 모델에 있으면 임베딩 벡터(np.ndarray)를 반환, 없으면 None 반환
def get_word_embedding(word):
    if word in model_wv:          # 사전학습 단어가 존재하면
        return model_wv[word]     # 해당 단어 임베딩 벡터 반환
    else:
        return None
    
# get_word_embedding('bad')

for word, index in word_to_index.items():  # 내 단어사전(단어-> 인덱스)를 순회
    if index >= 2:                         # 특수토큰 제외
        emb = get_word_embedding(word)     # 사전학습 임베딩에서 해당 단어 벡터 조회
        if emb is not None:                # 벡터가 존재하면
            embedding_matrix[index] = emb  # 내 인덱스 위치에 사전학습 벡터를 복사해서 채운다.

In [19]:
pd.DataFrame(embedding_matrix, index=word_to_index.keys())

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
<PAD>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
<UNK>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
nice,0.158203,0.105957,-0.189453,0.386719,0.083496,-0.267578,0.083496,0.113281,-0.104004,0.178711,...,-0.085449,0.189453,-0.146484,0.134766,-0.040771,0.032715,0.089355,-0.267578,0.008362,-0.213867
great,0.071777,0.208008,-0.028442,0.178711,0.132812,-0.099609,0.096191,-0.116699,-0.008545,0.148438,...,-0.011475,0.064453,-0.289062,-0.048096,-0.199219,-0.071289,0.064453,-0.167969,-0.020874,-0.142578
best,-0.126953,0.021973,0.287109,0.153320,0.127930,0.032715,-0.115723,-0.029541,0.153320,0.011292,...,0.006439,-0.033936,-0.166016,-0.016846,-0.048584,-0.022827,-0.152344,-0.101562,-0.090332,0.088379
amazing,0.073730,0.004059,-0.135742,0.022095,0.180664,-0.046631,0.224609,-0.229492,-0.040039,0.225586,...,0.018433,-0.021240,-0.250000,-0.020142,-0.310547,-0.207031,-0.006317,-0.141602,-0.150391,-0.137695
stop,-0.057861,0.013184,0.115234,0.069824,-0.306641,-0.044678,0.048584,0.152344,0.073242,-0.100098,...,0.100098,0.171875,-0.113281,0.064453,-0.115723,0.048096,-0.004822,0.086426,0.029907,0.007812
lies,0.149414,-0.012817,0.328125,0.025513,0.017334,0.190430,0.188477,-0.143555,-0.090820,0.206055,...,-0.308594,0.183594,-0.202148,0.031494,-0.164062,-0.201172,0.080078,-0.105469,0.149414,0.157227
pitiful,0.269531,0.253906,-0.020996,0.060303,-0.010925,0.217773,0.139648,-0.057617,0.312500,0.253906,...,-0.063477,0.132812,-0.094238,0.089355,-0.065430,-0.016235,-0.107910,-0.072266,-0.094238,0.028809
nerd,0.265625,-0.207031,-0.026611,0.419922,-0.208984,0.390625,0.164062,0.063965,0.149414,-0.017700,...,0.215820,0.125000,-0.227539,-0.310547,-0.112793,-0.096680,0.255859,0.124023,-0.030273,0.082031


In [20]:
# Pytorch 텍스트 분류 모델 : Embedding + RNN + Linear로 이진 분류(logit) 출력
import torch
import torch.nn as nn           # 신경망 레이어
import torch.optim as optim     # 옵티마이저(활성화함수)
from torch.utils.data import DataLoader, TensorDataset  # 배치 로더 / 데이터셋 유틸

class SimpleNet(nn.Module):
    # 정수 시퀀스를 임베딩 -> RNN -> 선형층으로 처리해 이진 분류 logit(1개)를 출력
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()                    # nn.Module 초기화
        self.embedding = nn.Embedding(        # 단어 ID를 밀집 벡터로 변환하는 임베딩 층
            num_embeddings = vocab_size,      # 단어 사전 크기 (어휘 수)
            embedding_dim = embedding_dim,    # 임베딩 차원
            padding_idx = 0                   # PAD(0) 인덱스는 0 그대로 사용
        )

        # 사전학습된 임베딩 벡터로 초기화 : Embedding 가중치를 사전학습 행렬로 덮어쓰기
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float))

        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)    # 입력(배치, 길이, 차원) 형태의 RNN
        self.out = nn.Linear(hidden_size, 1)     # 마지막 은닉 상태를 1차원 logit으로 변환

    def forward(self, x):
        embedded = self.embedding(x)     # (batch, seq_len) -> (batch, seq_len, embedding_dim)
        out, h_n = self.rnn(embedded)    # h_n: (num_layers*directions, batch, hidden_size)
        out = self.out(h_n.squeeze(0))   # (batch_size, hidden_size) -> (batch, 1)
        return out  # 출력 : 시그모이드 전 logit(확률이 아님)
    
embedding_dim = model_wv.vectors.shape[1]  # 사전학습 임베딩 차원 (300)으로 임베딩 차원 설정
model = SimpleNet(vocab_size, embedding_dim, hidden_size=16)  # 어휘 크기 / 임베딩 차원/ 은닉크기로 모델 생성
print(model)

criterion = nn.BCEWithLogitsLoss()  # 출력 logit과 정답(0/1)로 이진분류 손실 계산 (시그모이드 포함)
optimizer = optim.Adam(model.parameters(), lr=0.005)  # 모델 파라미터는 Adam으로 업데이트

SimpleNet(
  (embedding): Embedding(17, 300, padding_idx=0)
  (rnn): RNN(300, 16, batch_first=True)
  (out): Linear(in_features=16, out_features=1, bias=True)
)


In [21]:
# 학습 루프 : 미니배치 단위로 20 epoch 학습하며 평균 손실 출력
for epoch in range(20):
    epoch_loss = 0    # 손실 누적

    for x_batch, y_batch in dataloader:    # 미니배치 단위로 (X, y) 가져오기
        optimizer.zero_grad()              # 이전 배치 기울기 초기화
        output = model(x_batch)            # 순전파로 logit 계산
        loss = criterion(output, y_batch)  # 예측 logit과 정답으로 손실 계산
        loss.backward()                    # 역전파로 기울기 계산
        optimizer.step()                   # 파라미터 업데이트

        epoch_loss += loss.item()  # 배치손실을 float으로 누적
    
    print(f"Epoch {epoch + 1}: Loss {epoch_loss / len(dataloader)}")  # epoch별 평균 손실 계산

Epoch 1: Loss 0.7078307420015335
Epoch 2: Loss 0.5741119906306267
Epoch 3: Loss 0.45387016981840134
Epoch 4: Loss 0.3885459378361702
Epoch 5: Loss 0.289472796022892
Epoch 6: Loss 0.2057625986635685
Epoch 7: Loss 0.1449330635368824
Epoch 8: Loss 0.09873061440885067
Epoch 9: Loss 0.07171069271862507
Epoch 10: Loss 0.05469982326030731
Epoch 11: Loss 0.04191338876262307
Epoch 12: Loss 0.032103403471410275
Epoch 13: Loss 0.02679432090371847
Epoch 14: Loss 0.021011684788390994
Epoch 15: Loss 0.018954440485686064
Epoch 16: Loss 0.016619653441011906
Epoch 17: Loss 0.014636388281360269
Epoch 18: Loss 0.013677880400791764
Epoch 19: Loss 0.012031156802549958
Epoch 20: Loss 0.011078468756750226


In [22]:
# 평가 / 예측 : 학습된 모델로 확률 -> 0/1 예측값 생성 후 정답과 비교
model.eval()                        # 평가 모드
with torch.no_grad():               # 기울기 계산 비활성화
    output = model(X)               # 전체 샘플에 대한 예측 logit 계산
    prob = torch.sigmoid(output)    # logit에 0~1 확률로 변환
    pred = (prob >= 0.5).int()      # 임계값 0.5 기준으로 이진 분류(0/1) 예측값 생성

print(labels)
print(pred.squeeze().detach().numpy())  # 예측라벨을 1차원 numpy 배열로 변환

[1, 0, 0, 1, 1, 0, 1]
[1 0 0 1 1 0 1]


사전학습 임베딩을 사용했을 때에도 학습 데이터 분류가 잘 되는지 파악한다.  
만약 틀린 샘플이 있다면 해당 문장이 OOV(0벡터) 비중이 큰지 확인해봐야 한다.